In [1]:
!pip install datasets
from datasets import load_dataset

ds = load_dataset("cq01/mawps-asdiv-a_svamp")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
!pip install --upgrade pip

!pip uninstall unsloth -y
!pip uninstall torch torchvision -y
!pip uninstall xformers -y

!pip install "unsloth[cu124-torch250] @ git+https://github.com/unslothai/unsloth.git"
!pip install torch==2.4.1 torchvision==0.19.1
!pip install --upgrade xformers==0.0.28.post1

  Created wheel for unsloth: filename=unsloth-2024.11.7-py3-none-any.whl size=163138 sha256=2a766b0a95d5f9ce363993b85557b087770993479c929b39945a060f7dd907e3
  Stored in directory: /tmp/pip-ephem-wheel-cache-8ovelvf8/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d8342fd0d
Successfully built unsloth
  Attempting uninstall: triton
    Found existing installation: triton 3.0.0
    Uninstalling triton-3.0.0:
      Successfully uninstalled triton-3.0.0
  Attempting uninstall: nvidia-nvtx-cu12
    Found existing installation: nvidia-nvtx-cu12 12.1.105
    Uninstalling nvidia-nvtx-cu12-12.1.105:
      Successfully uninstalled nvidia-nvtx-cu12-12.1.105
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.20.5
    Uninstalling nvidia-nccl-cu12-2.20.5:
      Successfully uninstalled nvidia-nccl-cu12-2.20.5
  Attempting uninstall: nvidia-cusparse-cu12
    Found existing installation: nvidia-cusparse-cu12 12.1.0.106
    Uninstalling nvidia-cusparse

In [3]:
!python -m xformers.info
# !nvidia-smi

xFormers 0.0.28.post1
memory_efficient_attention.ckF:                    unavailable
memory_efficient_attention.ckB:                    unavailable
memory_efficient_attention.ck_decoderF:            unavailable
memory_efficient_attention.ck_splitKF:             unavailable
memory_efficient_attention.cutlassF-pt:            available
memory_efficient_attention.cutlassB-pt:            available
memory_efficient_attention.fa2F@v2.5.6-pt:         available
memory_efficient_attention.fa2B@v2.5.6-pt:         available
memory_efficient_attention.fa3F@0.0.0:             unavailable
memory_efficient_attention.fa3B@0.0.0:             unavailable
memory_efficient_attention.triton_splitKF:         unavailable
indexing.scaled_index_addF:                        unavailable
indexing.scaled_index_addB:                        unavailable
indexing.index_select:                             unavailable
sequence_parallel_fused.write_values:              available
sequence_parallel_fused.wait_values:       

In [4]:
import os
import torch
import numpy as np
import pandas as pd

In [5]:
for key, value in ds.items():
    print(f"Length of '{key}': {len(value)}")

Length of 'train': 3138
Length of 'validation': 1000


In [6]:
def generate_full_question(example):
    question = example["Question"]
    numbers = example["Numbers"]
    for i, num in enumerate(numbers):
        placeholder = f"number{i}"
        # Check if the number is an integer
        if num.is_integer():
            num_str = str(int(num))
        else:
            num_str = str(num)
        question = question.replace(placeholder, num_str)
    example["Full Question"] = question
    return example

# Apply the transformation to the train and validation sets
ds = ds.map(generate_full_question)

# Now you can access the transformed dataset with the new "Full Question" column
print(ds["train"][23]["Full Question"])

A construction company ordered 0.16666666666666666 ton of concrete , 0.16666666666666666 ton of bricks , and 0.5 ton of stone . How many tons of material did the company order in all ?


In [7]:
!pip install --upgrade accelerate einops
import torchvision
import torch
# !pip3 install -q --upgrade accelerate einops xformers

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)
from datasets import load_dataset
from peft import LoraConfig, PeftModel

In [8]:
# del trainer
# gc.collect()
# torch.cuda.empty_cache()

In [9]:
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

def load_model():
    """Load Gemma-2-9b model and tokenizer with LoRA applied."""
    max_seq_length = 2048
    load_in_4bit = True

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="unsloth/gemma-2-9b",
        max_seq_length=max_seq_length,
        dtype=None,
        load_in_4bit=load_in_4bit,
        device_map="auto",
    )

    model = FastLanguageModel.get_peft_model(
        model,
        r=16,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        lora_alpha=32,
        lora_dropout=0,
        bias="none",
        use_gradient_checkpointing="unsloth",
        random_state=3407,
        use_rslora=False,
        loftq_config=None
    )

    return model, tokenizer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [10]:
# Load the base Gemma-2-9b model
model, tokenizer = load_model()

==((====))==  Unsloth 2024.11.7: Fast Gemma2 patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2024.11.7 patched 42 layers with 42 QKV layers, 42 O layers and 42 MLP layers.


In [11]:
def format_answer(answer):
    """Convert answer to float format."""
    try:
        return f"the answer is {float(answer)}"
    except:
        # Fallback for any non-numeric answers
        return f"the answer is {answer}"

def create_few_shot_examples(num_shots=2):
    """Create few-shot examples with CoT reasoning."""
    examples = [
        {
            "question": "Sarah has 3 bags of marbles. Each bag contains 12 marbles. She gives 5 marbles to her friend. How many marbles does Sarah have now?",
            "reasoning": """the answer is 31.0""",
        },
        {
            "question": "A recipe calls for 2.5 cups of flour to make 1 dozen cookies. How many cups of flour are needed to make 3 dozen cookies?",
            "reasoning": """the answer is 7.5""",
        },
        {
            "question": "Tom has $100. He spends 40% of the money on books. How much money does he have left now?",
            "reasoning": """the answer is 60.0""",
        }
    ]
    return examples[:num_shots] if num_shots > 0 else []

def format_mawps_data(examples, num_few_shots=2):
    """Preprocess MAWPS-ASDIV-A dataset for training."""
    questions = examples["Full Question"]
    answers = examples["Answer"]
    texts = []
    few_shot_examples = create_few_shot_examples(num_few_shots)

    prompt_template = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Solve the following math problem. Provide only the numerical answer in the format: "the answer is [number]" without any explanation.

{few_shot_text}
### Input:
{input}

### Response:
{response}"""

    # Format few-shot examples if any
    few_shot_text = ""
    if few_shot_examples:
        few_shot_text = "\n\nHere are some examples for reference:\n\n"
        for ex in few_shot_examples:
            few_shot_text += f"### Input:\n{ex['question']}\n### Response:\n{ex['reasoning']}\n\n"

    for question, answer in zip(questions, answers):
        text = prompt_template.format(
            few_shot_text=few_shot_text,
            input=question,
            response=format_answer(answer)
        )
        texts.append(text)

    return {"text": texts}

# Process the dataset
formatted_dataset = ds.map(
    format_mawps_data,
    remove_columns=ds["train"].column_names,
    batched=True
)

In [12]:
def verify_formatting(dataset, num_examples=2):
    """Verify the formatting of the processed dataset."""
    print("Sample formatted examples:")
    for i in range(min(num_examples, len(dataset))):
        print(f"\nExample {i+1}:")
        print(dataset[i]["text"])
        print("-" * 50)

print("Dataset format verification:")
verify_formatting(formatted_dataset["validation"])

Dataset format verification:
Sample formatted examples:

Example 1:
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Solve the following math problem. Provide only the numerical answer in the format: "the answer is [number]" without any explanation.



Here are some examples for reference:

### Input:
Sarah has 3 bags of marbles. Each bag contains 12 marbles. She gives 5 marbles to her friend. How many marbles does Sarah have now?
### Response:
the answer is 31.0

### Input:
A recipe calls for 2.5 cups of flour to make 1 dozen cookies. How many cups of flour are needed to make 3 dozen cookies?
### Response:
the answer is 7.5


### Input:
julia played tag with 18 kids on monday . she played tag with 10 kids on tuesday . how many more kids did she play with on monday than on tuesday ?

### Response:
the answer is 8.0
----------------------------------------------

In [13]:
torch.cuda.empty_cache()
import gc
gc.collect()

25

In [14]:
# del trainer

In [15]:
# gc.collect()

In [16]:
training_config = TrainingArguments(
    output_dir="./math_model_output",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    # per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    logging_steps=10,
    save_strategy="epoch",
    seed=3407,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset["train"],
    # eval_dataset=formatted_dataset["validation"],
    tokenizer=tokenizer,
    args=training_config,
    dataset_text_field="text",
    max_seq_length=512,
    dataset_num_proc=1,
    packing=False,
)

In [17]:
torch.cuda.empty_cache()
gc.collect()

3687

In [18]:
print("Starting training...")

trainer.train()

# Save the final model
trainer.save_model("./math_model_final")

Starting training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,138 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 2
\        /    Total batch size = 16 | Total steps = 196
 "-____-"     Number of trainable parameters = 54,018,048
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: dassprojects2024 (dassprojects2024-idk). Use `wandb login --relogin` to force relogin


Step,Training Loss
10,0.597500
20,0.310600
30,0.290000
40,0.277100
50,0.269200
60,0.266500
70,0.270700
80,0.253900
90,0.249500
100,0.238900


In [23]:
model.save_pretrained("few-shot-prompt/models/few_shot_mathGemma-2-9b")
tokenizer.save_pretrained("few-shot-prompt/models/few_shot_mathGemma-2-9b")

('few-shot-prompt/models/few_shot_mathGemma-2-9b/tokenizer_config.json',
 'few-shot-prompt/models/few_shot_mathGemma-2-9b/special_tokens_map.json',
 'few-shot-prompt/models/few_shot_mathGemma-2-9b/tokenizer.model',
 'few-shot-prompt/models/few_shot_mathGemma-2-9b/added_tokens.json',
 'few-shot-prompt/models/few_shot_mathGemma-2-9b/tokenizer.json')

In [24]:
import shutil

shutil.make_archive("few-shot-prompt", 'zip', "few-shot-prompt")

'/content/few-shot-prompt.zip'

In [21]:
from google.colab import files

files.download("lmao.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
!ls
!pwd

huggingface_tokenizers_cache  lmao.zip		math_model_output  wandb
lmao			      math_model_final	sample_data
/content
